In [3]:
# Perturbation implementation of asset pricing model adapted from Fabrice Collard's Matlab code, http://fabcol.free.fr/
# tested in Julia 1.5.0
# this code is part of chapter 5, "Advanced Numerical Techniques" from the book: "Introduction to Quantitative Macroeconomics with Julia"
# Academic Press - Elsevier
# for comments, email at: petre(dot)caraiani(at)gmail(dot)com


#Pkg.add("ForwardDiff")
#Pkg.add("FiniteDiff")
#using ForwardDiff
#using FiniteDiff
using Calculus;
using LinearAlgebra;
using Plots;



#calibrate

nx  = 1;   # backward variables
ny  = 1;   # static variables
ne  = 1;   # shocks

rhox = -0.139;
eta  = 0.0348;
sx   = eta/sqrt(1-rhox*rhox);


beta  = 0.950;
theta = -1.500;
xb    = 0.0179;
rhox  = -0.139;
eta   = 0.0348;
ETA   = [eta];
s2    =  1;
sx    = eta/sqrt(1-rhox*rhox);
ys    = beta*exp(theta*xb)/(1-beta*exp(theta*xb));
param = [beta theta rhox xb];

# steady state

devx  = 5;
nbx   = 200;
xmin  = xb-devx*sx;
xmax  = xb+devx*sx;
xgrid = collect(LinRange(xmin,xmax,nbx))


xs=[xb ys xb ys];
xs=vec(xs)


4-element Array{Float64,1}:
  0.0179
 12.30351462782001
  0.0179
 12.30351462782001

In [4]:
#define the model
function model(xx,param);

beta  = param(1);
theta = param(2);
rhox  = param(3);
xb    = param(4);

xp    = xx(1);
yp    = xx(2);
x     = xx(3);
y     = xx(4);
    

#Backward variables
# k' a' c' k a c
# 1  2  3  4 5 6
eq1   = xp-rhox*x-(1-rhox)*xb;

# Forward variables

eq2   = y-beta*exp(theta*xp)*(1+yp);
############################################################################
# In case you want a log-quadratic approximation, use this version of the code

# k' a' c' k a c
# 1  2  3  4 5 6
#eq(1)	= exp(kp)-exp(a+alpha*k)+exp(c)-(1-delta)*exp(k); 
#eq(2)	= ap-ra*a-(1-ra)*ab;
#
# Forward variables
#
#eq(3)	= exp(-sigma*c)-beta*exp(-sigma*cp)*(alpha*exp(ap+(alpha-1)*kp)+1-delta);
end

model (generic function with 1 method)

In [5]:
#compute Jacobian and Hessian matrices
#Pkg.add("ForwardDiff")
#Pkg.add("FiniteDiff")
#using ForwardDiff
#using FiniteDiff
using Calculus

#define the model
# variables (leads and lags)
#xp = xx(1);
#yp = xx(2);
#x  = xx(3);
#y  = xx(4);

#Jacobian
gr1 = Calculus.gradient(x -> x[1]-rhox*x[3]-(1-rhox)*xb, xs);
gr2 = Calculus.gradient(x -> x[4]-beta*exp(theta*x[1])*(1+x[2]), xs);

#define the Jacobian
J =  [gr1'; gr2']

2×4 Array{Float64,2}:
  1.0      0.0       0.139  0.0
 18.4553  -0.924832  0.0    1.0

In [6]:
#Hessian
h1 = hessian(x -> x[1]-rhox*x[3]-(1-rhox)*xb, xs);
h2 = hessian(x -> x[4]-beta*exp(theta*x[1])*(1+x[2]), xs);

h1=vec(h1);
h2=vec(h2);
#define the Hessian
H =  [h1 h2]

16×2 Array{Float64,2}:
 0.0  -27.6829
 0.0    1.38725
 0.0    0.0
 0.0    0.0
 0.0    1.38725
 0.0   -4.08529e-9
 0.0    0.0
 0.0    0.0
 0.0    0.0
 0.0    0.0
 0.0    0.0
 0.0    0.0
 0.0    0.0
 0.0    0.0
 0.0    0.0
 0.0    4.07739e-8

In [7]:
#sort eigenvalues functions
#qzswitch procedure

function reorder(s,t,q,z);
nsize = size(s,1);
i = 1;

while i<=nsize-1;

   if 1+abs(t[i,i]*s[i+1,i+1])>1+abs(s[i,i]*t[i+1,i+1]);    
      A=s; B=t; Q=q; Z=z;   

      s,t,q,z = qzswitch(i,A,B,Q,Z);
              
   i=i+1;
   end;         
end;            
return s,t,q,z;  
end;

function qzswitch(i,A,B,Q,Z);
    a = A[i,i]; d = B[i,i]; b =A[i,i+1]; e = B[i,i+1]; c = A[i+1,i+1]; f = B[i+1,i+1]; 

      wz = [c*e-f*b, (c*d-f*a)'];
      xy = [(b*d-e*a)', (c*d-f*a)'];
      n = sqrt(wz'*wz);
      m = sqrt(xy'*xy);

      if n == 0
      return
      else
    wz = \(n,wz);
    xy = \(m,xy); 
    wz = [wz[1] wz[2]; -wz[2] wz[1]]
    xy = [xy[1] xy[2]; -xy[2] xy[1]]
    A[i:i+1,:] = xy*A[i:i+1,:];
    B[i:i+1,:] = xy*B[i:i+1,:];
    A[:,i:i+1] = A[:,i:i+1]*wz;
    B[:,i:i+1] = B[:,i:i+1]*wz;
    Z[:,i:i+1] = Z[:,i:i+1]*wz;
    Q[i:i+1,:] = xy*Q[i:i+1,:];
       end;

   if ~(i==1);i = i-2;end   
return A,B,Q,Z;    
end; 

In [20]:
#Solve the model

function solve(xs,J,H,ETA,nx,ny,ne);

Hs=xs[1:nx];
Gs=xs[nx+1:nx+ny];

# Order 1

J0=J[:,1:nx+ny];
J1=-J[:,nx+ny+1:end];

r = schur(J0,J1)

s = r.S
t = r.T
q = r.Q  # So now q*a*z = s and q*b*z = t
z = r.Z

s,t,q,z=reorder(s,t,q,z)
    
 z21 = z[nx+1:end,1:nx];
z11 = z[1:nx,1:nx];

if rank(z11)<nx;
 error("Invertibility condition violated")
end

z11i = \(z11,Matrix{Float64}(I, nx, nx));
s11 = s[1:nx,1:nx];
t11 = t[1:nx,1:nx];

#if abs(t[nx,nx])>abs(s[nx,nx]) | abs(t[nx+1,nx+1])<abs(s[nx+1,nx+1]);
#warning("Wrong number of stable eigenvalues");
#end 
    
#compute Gx and Hx
dyn = \(s11,t11);
Gx = z21*z11i;
Hx = z11*dyn*z11i;

tol=1e-6;
if maximum(maximum(imag(Gx)))<tol;
  Gx=real(Gx);
end
if maximum(maximum(imag(Gx)))<tol;
  Hx=real(Hx);
end;    
    
    
# Computes Gxx and Hxx

Zx = [Hx;Gx*Hx;Matrix{Float64}(I, nx, nx);Gx];
Jxp= J[:,1:nx];
Jyp= J[:,nx+1:nx+ny];
Jx = J[:,nx+ny+1:2*nx+ny];
Jy = J[:,2*nx+ny+1:2*(nx+ny)];
XX1 = [kron((Jxp+Jyp*Gx),Matrix{Float64}(I, nx*nx, nx*nx)) kron(Jyp,kron(Hx',Hx'))+kron(Jy,Matrix{Float64}(I, nx*nx, nx*nx))];
XX0 = -kron(Zx',Zx')*H;
XX0 = vec(XX0);
HGXX= \(XX1,XX0);
Hxx = HGXX[1:nx*nx*nx];
if maximum(maximum(imag(Hxx)))<tol
   Hxx=real(Hxx);
end
Gxx = HGXX[nx*nx*nx+1:end];
if maximum(maximum(imag(Gxx)))<tol
   Gxx=real(Gxx);
end;    

# Computes Gss and Hss

SS0= 0;
for i=1:ne;
    Zs    =[ETA[:,i];Gx*ETA[:,i];zeros(nx,1);zeros(ny,1)];
    TEMP0 = -kron(Zs',Zs')*H;
    TEMP0 = vec(TEMP0);
    TEMP1 = -Jyp*kron(Matrix{Float64}(I, ny, ny),kron(ETA[:,i]',ETA[:,i]'))*Gxx;
    TEMP1 = vec(TEMP1);
    SS0   = SS0.+TEMP0+TEMP1;
end

SS1  = [Jxp+Jyp*Gx (Jyp+Jy)];
HGSS = \(SS1,SS0);
Hss  = HGSS[1:nx];
Gss  = HGSS[nx+1:nx+ny];
if maximum(maximum(imag(Hss)))<tol
   Hss=real(Hss);
end
if maximum(maximum(imag(Gss)))<tol
   Gss=real(Gss);
end

Hxx = reshape(Hxx,nx*nx,nx)';
Gxx = reshape(Gxx,nx*nx,ny)';    
    
return Gs,Hs,Gx,Hx,Gxx,Hxx,Gss,Hss;
end;

In [23]:
#solution using solve
Gs,Hs,Gx,Hx,Gxx,Hxx,Gss,Hss = solve(xs,J,H,ETA,nx,ny,ne);
Gxx

1×1 Adjoint{Float64,Array{Float64,2}}:
 0.42052520108652125